In [1]:
from itertools import zip_longest
import argparse, sys, subprocess
from utils.common import exists
from os.path import basename
import gensim
import gzip
from gensim.models.keyedvectors import KeyedVectors
from gensim.utils import tokenize
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from time import time
import numpy as np
from chinese_whispers import chinese_whispers, aggregate_clusters
import codecs
import networkx as nx
from multiprocessing import Pool
from os.path import join
from Pyro4 import Proxy

from networkx_server import NetworxServerManager
import filter_clusters
import vector_representations.build_sense_vectors
from utils.common import ensure_dir
import pcz


verbose = True


def get_ego_network(ego):
    g = Proxy(uri)

    tic = time()
    ego_network = nx.Graph(name=ego)

    # Add related and substring nodes
    substring_nodes = []
    for j, node in enumerate(G_nodes):
        if ego.lower() == node.lower():
            node_nns = g.get_neighbors(node)["_atlas"]
            ego_nodes = [(rn, {"weight": node_nns[rn]["weight"]}) for rn in node_nns]
            ego_network.add_nodes_from(ego_nodes)
        else:
            if "_" not in node: continue
            if node.startswith(ego + "_") or node.endswith("_" + ego):
                substring_nodes.append( (node, {"weight": 0.99}) )
    ego_network.add_nodes_from(substring_nodes)

    # Find edges of the ego network
    for r_node in ego_network:
        related_related_nodes = g.get_neighbors(r_node)["_atlas"]
        related_related_nodes_ego = sorted(
            [(related_related_nodes[rr_node]["weight"], rr_node) for rr_node in related_related_nodes if rr_node in ego_network],
            reverse=True)[:n]
        related_edges = [(r_node, rr_node, {"weight": w}) for w, rr_node in  related_related_nodes_ego]
        ego_network.add_edges_from(related_edges)

    chinese_whispers(ego_network, weighting="top", iterations=20)
    if verbose: print("{}\t{:f} sec.".format(ego, time()-tic))

    g._pyroRelease()
    return ego_network

n = None
G_nodes = None 
uri = "PYRO:obj_f1f83063c74647da8af66b85da8132e5@localhost:41641" # None

def ego_network_clustering(neighbors_fpath, clusters_fpath, max_related=300, num_cores=32):
    global n
    global G_nodes 
    #G = NetworxServerManager(neighbors_fpath)
    g = Proxy(uri)
    G_nodes = g.get_node_list()
    
    with codecs.open(clusters_fpath, "w", "utf-8") as output, Pool(num_cores) as pool:
        output.write("word\tcid\tcluster\tisas\n")
        for i, ego_network in enumerate(pool.imap_unordered(get_ego_network, G_nodes)):
            if i % 100 == 0: print(i, "ego networks processed")
            sense_num = 1
            for label, cluster in sorted(aggregate_clusters(ego_network).items(), key=lambda e: len(e[1]), reverse=True):
                output.write("{}\t{}\t{}\t\n".format(
                    ego_network.name,
                    sense_num,
                    ", ".join( ["{}:{:.4f}".format(c_node, ego_network.node[c_node]["weight"]) for c_node in cluster] )
                ))
                sense_num += 1
    print("Clusters:", clusters_fpath)

neighbors_fpath = "model/text8.graph"
ego_network_clustering(neighbors_fpath, "model/tmp.clusters", num_cores=40)

Loading spacy model...


CommunicationError: connection to ('localhost', 41641) rejected: no free workers, increase server threadpool size

In [9]:
from networkx_server import NetworxServerManager

neighbors_fpath = "model/text8.graph"
G = NetworxServerManager(neighbors_fpath)

Loading the graph: model/text8.graph


BrokenPipeError: [Errno 32] Broken pipe


Loaded in 66.915203 sec.


Process NetworkX RPC server:
Traceback (most recent call last):
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/managers.py", line 749, in _callmethod
    conn = self._tls.connection
AttributeError: 'ForkAwareLocal' object has no attribute 'connection'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/panchenko/sensegram/networkx_server.py", line 49, in run_pyro_daemon
    d["uri"] = uri
  File "<string>", line 2, in __setitem__
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/managers.py", line 753, in _callmethod
    self._connect()
  File "/home/panchenko/anaconda/lib/python3.6/multiprocessing/managers.py", line 740, in _connect
    conn 

In [8]:
G.graph.get_neighbors("the")

CommunicationError: cannot connect to ('localhost', 46180): [Errno 111] Connection refused